In [1]:
import warnings
from datetime import datetime
import pyodbc
import pandas as pd
import os
import glob
import numpy as np
import shutil

warnings.filterwarnings('ignore')

cnxn = pyodbc.connect(driver = '{ODBC Driver 17 for SQL Server}', server = 'tcp:10.3.6.50', database = 'reports', trusted_connection = 'yes')

cursor = cnxn.cursor()

#####
#AX items
#####

query = '''

SELECT *

FROM SD_items_by_vendor_UPC2

'''

dfLHA = pd.read_sql(query, cnxn)


#####
#item groups
#####

query = '''

SELECT *

FROM SD_Item_Groups

'''

dfItemGroups = pd.read_sql(query, cnxn)

#####
#categories
#####

query = '''

SELECT *

FROM SD_AX_Item_Categories

'''

dfCategories = pd.read_sql(query, cnxn)


#####
#sections
#####

query = '''

SELECT *

FROM SD_store_sections

'''

dfSections = pd.read_sql(query, cnxn)


#####
#locations
#####

query = '''

SELECT *

FROM SD_inventory_locations

WHERE INVENTLOCATIONID NOT LIKE 'DAL%'

'''

dfLocations = pd.read_sql(query, cnxn)

print('Ready')

Ready


In [ ]:
dfLHA.drop(dfLHA.columns[[2,3,6,7,8,9,10,13,14,15]], axis = 1, inplace = True)
dfCategories.drop(dfCategories.columns[[1]], axis = 1, inplace = True)

dfItemGroups['ITEMGROUPID'] = dfItemGroups['ITEMGROUPID'].str.upper()

from functools import reduce

# compile the list of dataframes you want to merge
data_frames = [dfLHA, dfItemGroups, dfCategories]

df_merged = reduce(lambda left,right: pd.merge(left,right,on=['ITEMID'], how='left'), data_frames).fillna('null')

# df_merged['InventoryAvailable'] = df_merged['InventoryAvailable'].fillna(0)

df_merged['InventoryAvailable'] = df_merged['InventoryAvailable'].replace({'null':'0'})

df_merged['InventoryAvailable'] = pd.to_numeric(df_merged['InventoryAvailable'], downcast='integer')

cond1 = df_merged['InventoryAvailable'] > 0
cond2 = df_merged['ITEMGROUPID'] != 'MAP'
cond3 = df_merged['SubCategory'] != 'Cards'
cond4 = df_merged['SubSubCategory'] != 'Apparel/Accessories'
cond5 = df_merged['SubSubCategory'] != 'Soda'
cond6 = df_merged['SubSubCategory'] != 'Seasonal Candy'
cond7 = df_merged['SubSubCategory'] != 'Holiday'

df_merged = df_merged[cond1]

df_merged = df_merged[cond2]

df_merged = df_merged[cond3]

df_merged = df_merged[cond4 & cond5 & cond6 & cond7]

df_merged.drop_duplicates(subset = ['ITEMID', 'BarCode'], keep = False, inplace = True)

# compile the list of dataframes you want to merge
data_frames = [df_merged, dfSections]

df_merged_final = reduce(lambda left,right: pd.merge(left,right,on=['ITEMID'], how='outer'), data_frames).fillna('null')

df_merged_final = (df_merged_final[(df_merged_final['SECTIONID'] == 'null')])

# compile the list of dataframes you want to merge
data_frames = [df_merged_final, dfLocations]

df_merged_final_loc = reduce(lambda left,right: pd.merge(left,right,on=['ITEMID'], how='left'), data_frames).fillna('null')

df_merged_final_loc.drop_duplicates(subset = ['ITEMID', 'WMSLOCATIONID'], keep = False, inplace = True)

df_merged_final_loc.drop(df_merged_final_loc.columns[[5,8,9,10,11,12,14]], axis = 1, inplace = True)

df_merged_final_loc.rename(columns={'NAMEALIAS_x': 'NAMEALIAS', 'ITEMGROUPID': 'DEPT', 'WMSLOCATIONID': 'Location'}, inplace=True)

df_merged_final_loc.reset_index(drop=True, inplace=True)

df_merged_final_loc['SubCategory'] = df_merged_final_loc['SubCategory'].replace({'null':''})

myPath = 'C:/Users/USER/OneDrive - COMPANY/Merchandising Documents/AX Imports/Store/Sections/Output/'

myDate = pd.datetime.today().strftime('%Y_%m_%d %H_%M')

df_merged_final_loc.to_excel(myPath + myDate + ' ItemsNoSections.xlsx', index=False, header=True)

df_merged_final_loc.head()

In [55]:
myColumnCount = len(df_merged_final_loc.columns)

i = 0

while i < myColumnCount:
    col = df_merged_final_loc.columns[i]
    # Get column index position of column
    col_index = list(df_merged_final_loc.columns).index(df_merged_final_loc.columns[i])
    print(str(col_index) + ' - ' + col)
    i += 1

0 - ITEMID
1 - NAMEALIAS_x
2 - ExternalID
3 - BarCode
4 - Discontinued
5 - InventoryAvailable
6 - ITEMGROUPID
7 - SubCategory
8 - SubSubCategory
9 - SubSubSubCategory
10 - NAMEALIAS_y
11 - STOREID
12 - SECTIONID
13 - PhysicalInventory
14 - INVENTLOCATIONID
15 - WMSLOCATIONID
